# FHIR server commands in Python
- Before proceeding you must start a FHIR server. For default use [hapi-fhir-jpaserver-starter](https://github.com/hapifhir/hapi-fhir-jpaserver-starter). Enter mvn jetty:run from project in cmd to launch server
- [POST file and validate](#POST-file-and-validate)
- [PUT file with ID](#PUT-file-with-ID)
- [GET with validate](#GET-with-validate)
- [GET without validate - Download Resource](#GET-without-validate---Download-Resource)
- [Bulk Export](#Bulk-Export)

In [ ]:
import requests
import json
import ndjson
import os
import pandas as pd
from pathlib import Path
import time

In [ ]:
# resource lookup
file_to_resource = {
    'condition': 'Condition',
    'encounter': 'Encounter',
    'encounter_icu': 'Encounter',
    'medication': 'Medication',
    'medadmin_icu': 'MedicationAdministration',
    'medadmin': 'MedicationAdministration',
    'medication_request': 'MedicationRequest',
    'observation_chartevents': 'Observation',
    'observation_datetimeevents': 'Observation',
    'observation_labs': 'Observation',
    'observation_micro_org': 'Observation',
    'observation_micro_susc': 'Observation',
    'observation_micro_test': 'Observation',
    'observation_outputevents': 'Observation',
    'organization': 'Organization',
    'patient': 'Patient',
    'procedure': 'Procedure',
    'procedure_icu': 'Procedure'
}

In [ ]:
# read in one resource json
filename = 'encounter'
resource = file_to_resource[filename]


fhir_json = Path('/home/alex/Documents/mimic-fhir-test-data/V2/' + filename + '.json') 
count = 0 
with open(fhir_json, mode='r') as raw_fhir:
    for line in raw_fhir:
        count = count + 1
        fhir_data = json.loads(line)
        if count == 2:
            break

# Validate whole ndjson 
1. Validate fhir resources in ndjson file
2. Output resources that pass validation
3. Output issue statements for resources that fail validation

In [ ]:
def put_resource(resource, fhir_data):
    server = 'http://localhost:8080/fhir/'
    url = server +  resource + '/' + fhir_data['id']

    resp = requests.put(url,  json = fhir_data, headers={"Content-Type": "application/fhir+json"} )
    output = json.loads(resp.text)
    return output

def get_resource(resource, fhir_id):
    server = 'http://localhost:8080/fhir/'
    url = server +  resource + '/' + fhir_id
    fhir_resource = requests.get(url, headers={"Content-Type": "application/json"})

    output_json = output_dir / 'valid.json'
    with open(output_json, mode='a+') as output_fhir:
        json.dump(json.loads(fhir_resource.text), output_fhir)
        output_fhir.write('\n')

def output_issues(resource, fhir_id, outcome):
    id = fhir_data['id']
    output_json = output_dir / 'invalid.json'
    outcome['fhir_id'] = resource + '/' + fhir_id
    with open(output_json, mode='a+') as output_fhir:
        json.dump(outcome, output_fhir)
        output_fhir.write('\n')

In [ ]:
%%time
filename = 'encounter'
resource = file_to_resource[filename]
timestr = time.strftime("%Y%m%d-%H%M%S")
output_dir = Path('/home/alex/Documents/mimic-fhir-test-data/server_output/' + resource + '/' + timestr)
Path(output_dir).mkdir(parents=True, exist_ok=True)

fhir_json = Path('/home/alex/Documents/mimic-fhir-test-data/V_fail/' + filename + '.json') 
with open(fhir_json, mode='r') as raw_fhir:
    for line in raw_fhir:
        if line == '\n':
            continue
        else:
            fhir_data = json.loads(line)
            outcome = put_resource(resource, fhir_data)

            if outcome['resourceType'] == 'OperationOutcome':
                output_issues(resource, fhir_data['id'], outcome)            
            else:            
                get_resource(resource, fhir_data['id'])

## Terminology Expansion
- Custom codesystems and valuesets are not expanding properly
- Test individually posting and expanding them to get more debug info
- Need to post all mimic terminology separately or it won't fully expand... base expansion does not grab the full codesystems

In [ ]:
%%time
# CodeSystems
version = '0.1.1' # Need to change version to trigger expansion (does not need to be greater just different)
codesystems = ['admission-class', 'admission-type', 'admission-type-icu', 'admit-source', 
               'bodysite', 'd-items', 'd-labitems' 'diagnosis-icd9', 'diagnosis-icd10', 
               'discharge-disposition', 'lab-flags', 'medadmin-category-icu', 'medication-method',
               'medication-route', 'medication-site', 'microbiology-antibiotic', 
               'microbiology-interpretation', 'microbiology-organism', 'microbiology-test', 
               'mimic-observation-category', 'procedure-category', 'procedure-icd9', 
               'procedure-icd10', 'units']


for codesystem in codesystems:
    codesystem_file = Path('/home/alex/Documents/mimic-fhir-test-data/terminology/CodeSystem-' + codesystem + '.json') 
    with open(codesystem_file, mode='r') as cs_content:
        cs = json.load(cs_content)
        
    cs['version'] = version
    put_resource('CodeSystem',cs)
  

In [ ]:
%%time
# ValueSets

version = '0.1.1' # Need to change version to trigger expansion (does not need to be greater just different)
valuesets = ['admission-class', 'admission-type', 'admission-type-icu', 'admit-source', 
           'bodysite', 'chartevents-d-items', 'datetime-d-items', 'd-labitems'
           'diagnosis-icd', 'discharge-disposition', 'lab-flags', 'medadmin-category-icu', 
           'medication-method', 'medication-route', 'medication-site', 'microbiology-antibiotic',
           'microbiology-interpretation', 'microbiology-organism', 'microbiology-test', 
           'mimic-observation-category', 'outputevents-d-items', 'procedure-category', 
           'procedure-d-items', 'procedure-icd', 'units']

valuesets = ['chartevents-d-items']
for valueset in valuesets:
    valueset_file = Path('/home/alex/Documents/mimic-fhir-test-data/terminology/ValueSet-' + valueset + '.json') 
    with open(valueset_file, mode='r') as vs_content:
        vs = json.load(vs_content)
        
    vs['version'] = version
    put_resource('ValueSet',vs)

In [ ]:
%%time
# PUT single codesystem/valueset combo
filename = 'microbiology-test'
codesystem_file = Path('/home/alex/Documents/mimic-fhir-test-data/terminology/CodeSystem-' + filename + '.json') 
valueset_file = Path('/home/alex/Documents/mimic-fhir-test-data/terminology/ValueSet-' + filename + '.json') 

with open(codesystem_file, mode='r') as codesystem:
    cs = json.load(codesystem)
    
with open(valueset_file, mode='r') as valueset:
    vs = json.load(valueset)

cs['version'] = '0.1.1'
vs['version'] = '0.1.1'
output = put_resource('CodeSystem',cs)
output = put_resource('ValueSet',vs)
output   


In [ ]:
# Put just a valueset from file
# Motivation for this is that expansion by default won't expand large valuesets. But putting valueset directly it will (from forum  with James Agnew... https://groups.google.com/g/hapi-fhir/c/GCIl9RAbalw)
filename = 'microbiology-test'
valueset_file = Path('/home/alex/Documents/mimic-fhir-test-data/terminology/ValueSet-' + filename + '.json') 

with open(valueset_file, mode='r') as valueset:
    vs = json.load(valueset)

#output = put_resource('CodeSystem',cs)
output = put_resource('ValueSet',vs)
output   

In [ ]:
# Validate individual code
url = 'http://localhost:8080/fhir/ValueSet/' + vs['id'] +'/$validate-code?code=URGENET&system=http://mimic.fhir.mit.edu/CodeSystem/admission-class' 
validate_output = requests.get(url, headers={"Content-Type": "application/json"})
print(validate_output.text)


In [ ]:
# PUT valuest
url = 'http://localhost:8080/fhir/ValueSet/' + vs['id']  
put_output = requests.put(url, json=vs, headers={"Content-Type": "application/json"})
print(put_output.text)

In [ ]:
# GET valueset
url = 'http://localhost:8080/fhir/ValueSet/' + vs['id']   
get_result = requests.get(url, headers={"Content-Type": "application/json"})
print(get_result.text)

In [ ]:
# Expand valueset
url = 'http://localhost:8080/fhir/ValueSet/microbiology-test/$expand'  
expand_result = requests.get(url, headers={"Content-Type": "application/json"})
print(expand_result.text)

In [ ]:
# Validate codesystem
url = 'http://localhost:8080/fhir/ValueSet/$expand?url=http://fhir.mimic.mit.edu/ValueSet/microbiology-test'
validate_result = requests.get(url, headers={"Content-Type": "application/json"})
print(validate_result.text)

In [ ]:
# Delete valueset
server = 'http://localhost:8080/fhir/'
url = server +  'ValueSet/admission-class'

resp = requests.delete(url)
print(resp.text)

In [ ]:
server = 'http://localhost:8080/fhir/'
url = server +  'ValueSet/$expand?url=http://fhir.mimic.mit.edu/ValueSet/admission-class'

resp = requests.post(url, headers={"Content-Type": "application/fhir+json"} )
print(resp.text)

In [ ]:
# Validate code against valueset
url = 'http://localhost:8080/fhir/$validate-code?code=64891&system=http://fhir.mimic.mit.edu/CodeSystem/diagnosis-icd9'   
get_result = requests.get(url, headers={"Content-Type": "application/json"})
print(get_result.text)

In [ ]:
# Validate code against codesystem
url = 'http://localhost:8080/fhir/CodeSystem/diagnosis-icd9/$validate-code?code=64891'#&system=http://fhir.mimic.mit.edu/CodeSystem/microbiology-test'   
get_result = requests.get(url, headers={"Content-Type": "application/json"})
print(get_result.text)

# Bulk validation
Using ndjson format to validate resources in bulk

In [ ]:
class FhirBundle:
    def __init__(self,id):
        self.resourceType = 'Bundle'
        self.type = 'transaction'
        self.id = id
        self.entry = []
        
    def add_entry(self, resource, request):        
        new_request = {}
        new_request['method'] = request
        if request == 'POST':
            new_request['url'] = resource['resourceType']
        elif request == 'PUT':
            new_request['url'] = resource['resourceType'] +'/' + resource['id']
        else:
            raise Exception(f'Request {request}, is not currently supported')
        
        new_entry = {}
        new_entry['resource'] = resource
        new_entry['request'] = new_request   
        new_entry['fullUrl'] = resource['id']
        self.entry.append(new_entry)
    
    def to_json(self):
        return self.__dict__
    
    def request(self):
        url = 'http://localhost:8080/fhir/'
        #requests.post(url,  json = self.to_json(), headers={"Content-Type": "application/fhir+json"} )
        resp = requests.post(url,  json = self.to_json(), headers={"Content-Type": "application/fhir+json"} )
        output = json.loads(resp.text)
        return output

In [ ]:
%%time
b1.request()

In [ ]:
%%time
# pull in microbiology content into bundle for one patient

b_micro = FhirBundle('microbio-bundle')

filenames = ['observation_micro_test', 'observation_micro_org', 'observation_micro_susc']
for filename in filenames:
    resource = file_to_resource[filename]
    fhir_json = Path('/home/alex/Documents/mimic-fhir-test-data/V2/' + filename + '.json') 
    
    with open(fhir_json, mode='r') as raw_fhir:
        for line in raw_fhir:
            if line == '\n':
                continue
            else:
                fhir_data = json.loads(line)
                b_micro.add_entry(fhir_data, 'PUT')

# send full microbio bundle        
output = b_micro.request()
#output

In [ ]:
output

In [ ]:
%%time
b1 = FhirBundle('new-bundle')

filename = 'condition'
resource = file_to_resource[filename]
fhir_json = Path('/home/alex/Documents/mimic-fhir-test-data/V2/' + filename + '.json') 

count = 0
with open(fhir_json, mode='r') as raw_fhir:
    for line in raw_fhir:
        count= count + 1
        if count%30 == 0:
            #break
            b1.request()
            b1 = FhirBundle('new-bundle')
        if line == '\n':
            continue
        else:
            fhir_data = json.loads(line)
            b1.add_entry(fhir_data, 'PUT')

# send request for remaining            
output = b1.request()
#output

In [ ]:
server = 'http://localhost:8080/fhir/'
url = server

resp = requests.post(url,  json = b1.to_json(), headers={"Content-Type": "application/fhir+json"} )
output = json.loads(resp.text)
output

In [ ]:
%%time
filename = 'encounter'
resource = file_to_resource[filename]

fhir_json = Path('/home/alex/Documents/mimic-fhir-test-data/V2/' + filename + '.json') 
count = 0
with open(fhir_json, mode='r') as raw_fhir:
    for line in raw_fhir:
        count= count+1
        if count == 500:
            break
        if line == '\n':
            continue
        else:
            fhir_data = json.loads(line)
            outcome = put_resource(resource, fhir_data)

In [ ]:
%%time
filename = 'encounter'
resource = file_to_resource[filename]
fhir_list = []

fhir_json = Path('/home/alex/Documents/mimic-fhir-test-data/V2/' + filename + '.json') 
count = 0
with open(fhir_json, mode='r') as raw_fhir:    
    for line in raw_fhir:
        count = count + 1
        if count == 300:
            break
        if line == '\n':
            continue
        else:
            fhir_list.append(json.loads(line))
            outcome = put_resource(resource, fhir_data)

## POST file and validate
- use to validate a resource and get the response right back
- will autogenerate an ID for storing the resource

In [ ]:
# validate resource
fhir_option = '/$validate'
server = 'http://localhost:8080/fhir/'
url = server +  resource + fhir_option

resp = requests.post(url, json = fhir_data, headers={"Content-Type": "application/fhir+json"})
print(resp.text)

## PUT file with ID
- Uploads the resource to the FHIR Server with ID generated in Postgres
- This will check references in resources, so may toss errors if the referenced resources haven't been uploaded yet
- PUT responses:
  - With issue: an OperationOutcome with the detailed issues
  - Without issue: a json representation of the resource

In [ ]:
# validate resource
fhir_option = '/$validate'
server = 'http://localhost:8080/fhir/'
url = server +  resource + '/' + fhir_data['id']

resp = requests.put(url,  json = fhir_data, headers={"Content-Type": "application/fhir+json"} )
print(resp.text)

## GET with validate
- grab resource by ID and validate it
- Resource must have been PUT on the server before this for it to work
- The advantage of using a GET validate versus a POST validate, is that the GET validate will have the text.narrative automatically generated after the PUT call. The narrative is a warning that pops up in the POST.

In [ ]:
url = server +  resource + '/' + fhir_data['id'] + fhir_option
resp = requests.get(url, headers={"Content-Type": "application/fhir+json"})
print(resp.text)

## GET without validate - Download Resource
- Download an individual resource, best to be done after validation complete

In [ ]:
url = server +  resource + '/' + fhir_data['id']
fhir_resource = requests.get(url, headers={"Content-Type": "application/json"})
print(fhir_resource.text)


output_json = Path('/home/alex/Documents/mimic-fhir-test-data/server_output/' + filename + '.json') 
with open(output_json, mode='a+') as output_fhir:
    #json.dump(json.loads(fhir_resource.text), output_fhir)
    #output_fhir.write('\n')
    output_fhir.write(f'{fhir_resource.text}')

## Bulk Export
- Can export all entries of a particular resource
- May need to enable bulk export for FHIR server: In application.yaml for HAPI FHIR Server need to set parameter `bulk_export_enabled` to true. Restart server to activate this parameter

TO DO:
- Figure out how to poll the bulk export endpoint: http://www.hl7.org/fhir/async.html and https://hl7.org/fhir/uv/bulkdata/export/index.html
- Find the output location
- Output a resource to a file

In [ ]:
url = server +  '$export?_type=' + resource
resp = requests.get(url, headers={"Accept": "json", "Prefer": "respond-async"})
print(resp.text)